# BM25 Algorithm
BM25 is a probabilistic algorithm for information retrieval that is described in [1]. It is summarized here.

## 1. Msmarco Dataset
We'll use the [msmarco](https://microsoft.github.io/msmarco/TREC-Deep-Learning-2019) dataset for examples in this notebook.

In [3]:
# msmarco-docs.tsv not uploaded to github due to size.
# !head ../../data/msmarco/msmarco-docs.tsv -n 1

D1555982	https://answers.yahoo.com/question/index?qid=20071007114826AAwCFvR	The hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.?	Science & Mathematics Physics The hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.? It is a good approximation to assume that the emissivity e is equal to 1 for these surfaces. Find the radius of the star Rigel, the bright blue star in the constellation Orion that radiates energy at a rate of 2.7 x 10^32 W and has a surface temperature of 11,000 K. Assume that the star is spherical. Use σ =... show more Follow 3 answers Answers Relevance Rating Newest Oldest Best Answer: Stefan-Boltzmann law states that the energy flux by radiation is proportional to the forth power of the temperature: q = ε · σ · T^4 The total energy flux at a spherical surface of Radius R is Q = q·π·R² = ε·σ·T^4·π·R² Hence the radius is R = √ ( Q / (ε·σ·T^4·π) ) = √ ( 2.7x10+32 W / (1 · 5.67x10-8W/m²K^4 · (1100K)^4 · π) 

Each line of the `msmarco-docs.tsv` contains one document, with four fields per document: document ID, URL, title, and body.

The `msmarco-docs.tsv` file contains 3.2 million documents in one 22GB file, which is cumbersome to work with.

In [1]:
# msmarco-docs.tsv not uploaded to github due to size.
# !wc ../../data/msmarco/docs/msmarco-docs.tsv -l

3213835 ../data/msmarco/msmarco-docs.tsv


The Linux *split* command will split the TSV file into 64 documents, each containing 50,000 documents and several hundred MB in size.

In [5]:
#! split -d -l 50000 ../data/msmarco/msmarco-docs.tsv
# ! ls -l -h | head

total 22G
-rw-rw-r-- 1 ubuntu ubuntu  24K May 25 00:18 bm25.ipynb
drwxrwxr-x 2 ubuntu ubuntu 4.0K May 24 04:12 wordpiece
-rw-rw-r-- 1 ubuntu ubuntu 339M May 24 03:46 x00
-rw-rw-r-- 1 ubuntu ubuntu 347M May 24 03:46 x01
-rw-rw-r-- 1 ubuntu ubuntu 337M May 24 03:46 x02
-rw-rw-r-- 1 ubuntu ubuntu 342M May 24 03:46 x03
-rw-rw-r-- 1 ubuntu ubuntu 335M May 24 03:46 x04
-rw-rw-r-- 1 ubuntu ubuntu 340M May 24 03:46 x05
-rw-rw-r-- 1 ubuntu ubuntu 345M May 24 03:46 x06


## 2. Simple IDF-ish Algorithm
### A. Math
Eq (1)
$$P(\textrm{rel}|d, q) \propto_q \frac{P(\textrm{rel}|d, q)}{P(\overline{\textrm{rel}}|d, q)}$$

$P(\textrm{rel}|d, q)$ is the probability that document $d$ is relevant for query $q$. Eq(1) shows that the probability is proportional to the odds ratio, which is the probability that the document is relevant over the probability that the document is not relevant.

Next, a lot of math and approximations happen and we end up with Eq (2). See section 2.4 of [1] for details.

Eq(2)
$$P(\textrm{rel}|d, q) \propto_q \sum_{\textbf{q}, tf_i > 0} w_i$$

$w_i$ is the weight for term $t_i$, which represents the importance of term $i$ in document $d$. We calculate the score of a document by summing the weights of all terms that appear in both the query $q$ and the document. We can then return the top $m$ documents in descending order of $P(\textrm{rel}|d, q)$.

A simple way of determining $w_i$ is to use Eq(3), which corresponds to equation 3.3 in [1]. This approach is very similar to a classical *idf* approach.

Eq(3)
$$w_i^{\textrm{IDF}} = \log\frac{N - n_i + 0.5}{n_i + 0.5}$$

N is the number of documents in the corpus and $n_i$ is the number of documents in the corpus that contain term $t_i$. For terms that appear in most documents, $n_i$ will be close to $N$ and $w_i^{\textrm{IDF}}$ will be close to zero. The values of $0.5$ in the numerator and denominator smooth the weights in the event of rare query terms.

There are about 3.2 million documents in the corpus. Working with a single 22GB file is problematic. Splitting it into files with 50,000 docs each.

### B. Information Retrieval Toy Example

In [2]:
import math

import nltk
from nltk.corpus import stopwords
import pandas as pd
import pickle
from tokenizers import BertWordPieceTokenizer

We will index and create a retrieval system for a corpus of 1000 documents selected from an arbitrary text file.

In [9]:
# x10.1 file contains 5000 documents (5000 rows)
doc_list = pd.read_csv("x10.1", sep="\t", header=None, names=["doc_id", "url", "title", "doc"])
doc_list

,doc_id,url,title,doc
0,D2273143,https://www.allaboutcircuits.com/textbook/refe...,Solving Simultaneous Equations,Solving Simultaneous Equations Chapter 4 - Alg...
1,D1029983,http://xynyth.com/resource/icemelter-concrete/...,.,"Icemelters and Concrete- The basics, what ever..."
2,D312314,http://medical-dictionary.thefreedictionary.co...,nurse's aide,"nurse's aide Also found in: Dictionary, Thesau..."
3,D582292,https://www.business-case-analysis.com/cost-al...,Cost Allocation and Cost Apportionment Definit...,Home > Encyclopedia > C > Cost Allocation Cost...
4,D1021364,http://mexicanspanish.com/equis/,Equis,"Wednesday, January 27, 2016 by Mark Robert Ale..."
...,...,...,...,...
4995,D2376648,http://surnames.meaning-of-names.com/shirazi/,.,Shirazi Meaning & Surname Resources Etymology ...
4996,D2404743,http://www.answers.com/Q/What_is_the_Military_...,How do you edit and cut off email draft?,Min0619 2 Contributions How do you edit and cu...
4997,D3239066,http://www.rhythm-in-music.com/introduction-wh...,What Is Rhythm?,Learn about Rhythm Home - Fundamentals Sound W...
4998,D3369056,http://www.dictionaryofengineering.com/definit...,.,Nearby Termsvirtual devicevirtual device drive...


We would like to remove stopwords form our index and queries.

In [8]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

This class creates a simple inverse document frequency index and provides the ability to search the index.

In [5]:
class IdfIndex():
    """Creates a term index based on inverse document frequency (IDF)
    
    The fomat of self.index is [n, weight, set(doc_ids)]
    n is the number of documents that contain the search term.
    """
    def __init__(self, doc_list):
        self.doc_list = doc_list
        self.index = {} # term: [n, weight, set(doc_ids)]
        self.tokenizer = BertWordPieceTokenizer(
            "wordpiece/bert-base-uncased-vocab.txt", lowercase=True)
        self.N = 0
        self.stopwords = set(stopwords.words('english'))
        self.stopwords.add("[CLS]")
        self.stopwords.add("[SEP]")
        
        self.add_docs()
        
    def add_docs(self):
        for row in self.doc_list.itertuples(False):
            self.add_doc(row[0], row[3])
        self.update_weights()
        
    def add_doc(self, doc_id, doc):
        """Adds a single document to the index.
        
        Since the index stores the number of matching documents,
        addition documents can be added to the index at any
        time.
        """
        if not isinstance(doc, str):
            return
        encoding = self.tokenizer.encode(doc)
        tokens = set(encoding.tokens)
        for tkn in tokens:
            self._add_term(tkn, doc_id)
        self.N += 1
            
    def _add_term(self, term, doc_id):
        """Adds a single term to the index.
        """
        if term in self.stopwords:
            return
        if term in self.index:
            if doc_id not in self.index[term][2]:
                self.index[term][0] += 1
                self.index[term][1] = None
                self.index[term][2].add(doc_id)
        else:
            self.index[term] = [1, None, set([doc_id])]
            
    def _calc_weight(self, n):
        """Calculates an IDF weight for a term using Eq(3)."""
        return math.log((self.N - n + 0.5)/(n + 0.5))
    
    def update_weights(self):
        """Updates all weights in the index."""
        for entry in self.index.values():
            entry[1] = self._calc_weight(entry[0])
        self.weight_df = pd.DataFrame(
            [{"term": key, "count": val[0], "weight": val[1]}
             for key, val in self.index.items()])
        
    def search(self, query):
        """Accepts a search query and returns matching documents."""
        # Break query into WordPiece tokens
        query_tokens = [tkn for tkn in self.tokenizer.encode(query).tokens
                       if tkn not in self.stopwords]
        token_doc_pairs = []

        # Emit a doc ID, weight, and corresponding token for each
        #   document with a term that occurs in the query.
        #   (Map phase of mapReduce)
        for tkn in query_tokens:
            for doc_id in self.index[tkn][2]:
                token_doc_pairs.append([doc_id, self.index[tkn][1], [tkn]])
                
        token_doc_pairs.sort(key=lambda x: x[0])
        
        # Combine weights for same documents
        #   (Combine phase)
        results = []
        current_pair = token_doc_pairs[0]
        for next_pair in token_doc_pairs[1:]:
            if next_pair[0] == current_pair[0]:
                current_pair = [current_pair[0], current_pair[1] + next_pair[1],
                               current_pair[2] + next_pair[2]]
            else:
                results.append(current_pair)
                current_pair = next_pair
        results.append(current_pair)
        
        # Put results in a Dataframe
        results.sort(key=lambda x: x[1], reverse=True)
        for result in results:
            result.append(
                self.doc_list.loc[self.doc_list.doc_id == result[0],
                                  "title"].values[0])
        results_df = pd.DataFrame(results, columns=["Doc ID",
                                                    "Summed Weights",
                                                    "Matching Terms",
                                                    "Title"])
        return results_df

Creating the index from the document list.

In [7]:
idf = IdfIndex(doc_list)

Exception: Error while initializing WordPiece: No such file or directory (os error 2)

A simple query

In [ ]:
idf.search("apple watch tips").head(10)

* The number of times a term appears in a document has no impact on the search results.
* A matching term contributes the same weight, no matter what document it appears in.

## 3. Adding Term Frequency

### 1. Math
For our next iteration of a weight equation, we'll use:

$$w_i(\textit{tf}) = \frac{\textit{tf}}{k + \textit{tf}} \cdot w^{IDF}$$

* $\textit{tf}$ is the number of occurrences of term $t$ in document $d$. Note that $\textit{tf}$ is *not* normalized by dividing by the number of words in a document. $\textit{tf}$ is a positive integer.
* $k$ is a chosen parameter. Robertson and Zaragoza report that values between 1.2 and 2.0 work well.
* As $\textit{tf}$ increases, the value $\frac{\textit{tf}}{k + \textit{tf}}$ approaches 1 asymptotically.  Weights are penalized for having only 1 or a few occurrences of a term, but they do not increase without bound as term frequency increases. Robertson and Zaragoza call this *saturation*.

In section 2, there was only one weight for a term, regardless of how many documents it appeared in, because the weight did not depend on how many times a term appeared in a document. With term frequency, there is a different weight for every term-document combination>

### 2. Toy Example

In [ ]:
class tfIndex(IdfIndex):
    """Index class that incorporates term frequency."""
    
    def __init__(self, doc_list, k):
        """Constructor
        Args:
            doc_list: Pandas dataframe of doc_id, url, title, and body.
            k: parameter used to modify term frequency.
        
        """
        self.k = k
        super().__init__(doc_list)
    
    def add_doc(self, doc_id, doc):
        """Adds a single document to the index.
        
        Since the index stores the number of matching documents,
        addition documents can be added to the index at any
        time.
        """
        if not isinstance(doc, str):
            return
        encoding = self.tokenizer.encode(doc)
        tkn_counts = {}
        for tkn in encoding.tokens:
            if tkn in tkn_counts:
                tkn_counts[tkn] += 1
            else:
                tkn_counts[tkn] = 1

        for tkn, ct in tkn_counts.items():
            self._add_term(tkn, ct, doc_id)
        self.N += 1
        
    def _add_term(self, tkn, ct, doc_id):
        """Adds a single term to the index.
        
        index format: {term: [n, set((doc_id, ct))]}
        """
        if tkn in self.stopwords:
            return
        if tkn in self.index:
            self.index[tkn][0] += 1
            self.index[tkn][1][doc_id] = [ct, None]
        else:
            self.index[tkn] = [1, {doc_id: [ct, None]}]
            
    def update_weights(self):
        for tkn in self.index.values():
            n = tkn[0]
            w_idf = math.log((self.N - n + 0.5)/(n + 0.5))
            for doc_id, val in tkn[1].items():
                tf = val[0]
                w_i = w_idf * tf / (self.k + tf)
                val[1] = w_i
                
    def search(self, query):
        """Accepts a search query and returns matching documents."""
        # Break query into WordPiece tokens
        query_tokens = [tkn for tkn in self.tokenizer.encode(query).tokens
                       if tkn not in self.stopwords]
        token_doc_pairs = []

        # Emit a doc ID, weight, and corresponding token for each
        #   document with a term that occurs in the query.
        #   (Map phase of mapReduce)
        for tkn in query_tokens:
            n = self.index[tkn][0]
            for doc_id, val in self.index[tkn][1].items():
                token_doc_pairs.append([doc_id, val[1], [tkn]])
                
        token_doc_pairs.sort(key=lambda x: x[0])
        
        # Combine weights for same documents
        #   (Combine phase)
        results = []
        current_pair = token_doc_pairs[0]
        for next_pair in token_doc_pairs[1:]:
            if next_pair[0] == current_pair[0]:
                current_pair = [current_pair[0], current_pair[1] + next_pair[1],
                               current_pair[2] + next_pair[2]]
            else:
                results.append(current_pair)
                current_pair = next_pair
        results.append(current_pair)
        
        # Put results in a Dataframe
        results.sort(key=lambda x: x[1], reverse=True)
        for result in results:
            result.append(
                self.doc_list.loc[self.doc_list.doc_id == result[0],
                                  "title"].values[0])
        results_df = pd.DataFrame(results, columns=["Doc ID",
                                                    "Summed Weights",
                                                    "Matching Terms",
                                                    "Title"])
        return results_df

Create a term-frequency index and show a few entries. Set k to 1.5.

The format of index entries is `{token: [doc_count, {doc_id: [term_count, term_weight]}]}`

In [ ]:
tf = tfIndex(doc_list.iloc[:100, :], 1.5)
list(tf.index.items())[50:55]

Try out a search.

In [ ]:
tf.search("apple watch tips").head(10)

Adding term frequency helps. The relevant document is not the first item returned, with a much higher summed weight.

## 4. Accounting for Document Length - Simple Version of BM25
### A. Math
Term frequency *tf* can be impacted by the length of a document. As explained in [1], if we don't adjust for length, verbose documents that use more words to deliver the same content will be penalized. However documents may be longer due to having a larger scope (covering more topics) instead of greater verbosity. With the BM25 algorithm, a document length normalization parameter is calculated as such:
$$ B := \left( (1 - b) + b \frac{\textit{dl}}{\textit{avedl}}\right)$$

* $b$ is a parameter chosen by the user. Robertson and Zaragoza report that values between 0.5 and 0.8 work well.
* $\textit{dl}$ is the length of a document.
* $\textit{avedl}$ is the average length of a document in the corpus.

Next, the term frequencies are adjusted for document length:

$$ \textit{tf}^\prime = \frac{\textit{tf}}{B}$$

Finally:
$$ w_i^{BM25} = \frac{\textit{tf}^\prime}{k + \textit{tf}^\prime} \cdot w_i^{IDF}$$

### B. Toy Example

In [ ]:
class Bm25Index(tfIndex):
    def __init__(self, doc_list, k, b):
        """Constructor
        Args:
            doc_list: Pandas dataframe of doc_id, url, title, and body.
            k: parameter used to modify term frequency.
        
        """
        self.b = b
        self.avedl = 0
        self.doc_lengths = {}
        super().__init__(doc_list, k)
        
    def add_doc(self, doc_id, doc):
        """Adds a single document to the index.
        
        Since the index stores the number of matching documents,
        addition documents can be added to the index at any
        time.
        """
        if not isinstance(doc, str):
            return
        encoding = self.tokenizer.encode(doc)
        tkn_counts = {}
        for tkn in encoding.tokens:
            if tkn in tkn_counts:
                tkn_counts[tkn] += 1
            else:
                tkn_counts[tkn] = 1

        for tkn, ct in tkn_counts.items():
            self._add_term(tkn, ct, doc_id)
        self.N += 1
        
        #### New line
        # Track document lengths
        self.doc_lengths[doc_id] = len(encoding.tokens)
        
    def update_weights(self):
        # Calcualate average document length
        self.avedl = sum(self.doc_lengths.values())/self.N  ## New Line
        
        for tkn in self.index.values():
            n = tkn[0]
            w_idf = math.log((self.N - n + 0.5)/(n + 0.5))
            for doc_id, val in tkn[1].items():
                tf = val[0]
                dl = self.doc_lengths[doc_id]
                B = (1 - self.b) + self.b * dl / self.avedl
                tf_prime = tf / B
                w_i_BM25 = w_idf * tf_prime / (self.k + tf_prime)
                val[1] = w_i_BM25

In [ ]:
tf = Bm25Index(doc_list.iloc[:5000, :], 1.5, 0.7)
list(tf.index.items())[50:52]

In [ ]:
tf.search("apple watch tips").head(10)

In [ ]:
tf.search("solving math equations").head(10)

## 5. Conclusion and Notes
This version of BM25 assumes no relevance data is available. If relevance data is available, Eq(3) is replaced by:

Eq(4)
$$ w_i^{RSJ} = \log \frac{(r_i + 0.5)(N - R - n_i + r_i + 0.5)}{(n_i - r_i + 0.5)(R - r_i + 0.5)}$$

* R is the total number of documents that are judged to be relevant to the query.
* r_i is the number of relevant documents that contain term $t_i$.
* N is the total number of documents in the corpus.
* n_i is the number of documents in the corpus that contain term $t_i$.

If the number of relevant documents in the corpus is small compared to the total number of documents, then Eq(4) can be approximated with Eq(3).

The BM25 algorithm here is a bit simpler than what was originally published. See section 3.5.1 of [1] for a description of the differences.

This example will not and is not meant to scale to the entire Msmarco dataset. It's purpose is to illustrate how a probabilistic IR retrieval algorithm works.

For generating indices on large corpora, a MapReduce framework like Spark may be useful.

## Citations
[1] Stephen E. Robertson and Hugo Zaragoza. 2009. The Probabilistic Relevance
Framework: BM25 and Beyond. [Foundations and Trends in Information Retrieval
(2009).](https://drive.google.com/file/d/1ni_fbufB4irOJTIRzunNQFD23MotEpT7/view?usp=sharing)